In [1]:
import exchange_calendars as xcals
import pandas as pd
from IPython.display import Markdown, display
from openbb import obb
from sqlalchemy import create_engine, text
from sqlalchemy.exc import ProgrammingError

In [3]:
obb.user.preferences.output_type = "dataframe"

In [11]:
username = "postgres"
password = "Kj4*spahxBPuZ!J"
host = "127.0.0.1"
port = "5432"
database = "market_data"

In [17]:
DATABASE_URL = f"postgresql://{username}:{password}@{host}:{port}/postgres"
base_engine = create_engine(DATABASE_URL)

In [19]:
def create_database_and_get_engine(db_name, base_engine):
    conn = base_engine.connect()
    conn = conn.execution_options(isolation_level="AUTOCOMMIT")

    try:
        conn.execute(text(f"CREATE DATABASE {db_name};"))
    except ProgrammingError:
        pass
    finally:
        conn.close()

    conn_str = base_engine.url.set(database=db_name)

    return create_engine(conn_str)

In [21]:
engine = create_database_and_get_engine("stock_data", base_engine)

In [23]:
def get_stock_data(symbol, start_date=None, end_date=None):
    data = obb.equity.price.historical(
        symbol,
        start_date=start_date,
        end_date=end_date,
        provider="yfinance",
    )
    data.reset_index(inplace=True)
    data["symbol"] = symbol
    return data

In [25]:
def save_data_range(symbol, engine, start_date=None, end_date=None):
    data = get_stock_data(symbol, start_date, end_date)
    data.to_sql("stock_data", engine, if_exists="append", index=False)

In [27]:
def save_last_trading_session(symbol, engine):
    today = pd.Timestamp.today()
    data = get_stock_data(symbol, today, today)
    data.to_sql("stock_data", engine, if_exists="append", index=False)

In [35]:
for symbol in ["SPY", "QQQ", "DIA"]:
    save_data_range(
        symbol, engine=engine, start_date="2020-06-01", end_date="2025-03-23"
    )

In [37]:
df_1 = pd.read_sql_query("SELECT * from stock_data where symbol='SPY'", engine)
display(df_1)

,date,open,high,low,close,volume,dividend,symbol
0,2020-06-01,303.619995,306.209991,303.059998,305.549988,55758300,0.000,SPY
1,2020-06-02,306.549988,308.130005,305.100006,308.079987,74267200,0.000,SPY
2,2020-06-03,310.239990,313.220001,309.940002,312.179993,92567600,0.000,SPY
3,2020-06-04,311.109985,313.000000,309.079987,311.359985,75794400,0.000,SPY
4,2020-06-05,317.230011,321.269989,317.160004,319.339996,150524700,0.000,SPY
...,...,...,...,...,...,...,...,...
1857,2025-03-17,562.789978,569.710022,562.349976,567.150024,49008700,0.000,SPY
1858,2025-03-18,564.799988,565.020020,559.059998,561.020020,66041400,0.000,SPY
1859,2025-03-19,562.830017,570.950012,561.630005,567.130005,66556000,0.000,SPY
1860,2025-03-20,563.330017,570.570007,562.599976,565.489990,62958200,0.000,SPY


In [39]:
df_2 = pd.read_sql_query(
    "SELECT * from stock_data where symbol='SPY' and volume > 100000000", engine
)
display(df_2)

,date,open,high,low,close,volume,dividend,symbol
0,2020-06-05,317.230011,321.269989,317.160004,319.339996,150524700,0.000,SPY
1,2020-06-11,311.459991,312.149994,300.010010,300.609985,209243600,0.000,SPY
2,2020-06-12,308.239990,309.079987,298.600006,304.209991,194678900,0.000,SPY
3,2020-06-15,298.019989,308.279999,296.739990,307.049988,135782700,0.000,SPY
4,2020-06-16,315.480011,315.640015,307.670013,312.959991,137627500,0.000,SPY
...,...,...,...,...,...,...,...,...
356,2024-04-19,499.440002,500.459991,493.859985,495.160004,102129100,0.000,SPY
357,2024-08-05,511.640015,523.580017,510.269989,517.380005,146267400,0.000,SPY
358,2024-12-18,603.979980,606.409973,585.890015,586.280029,108248700,0.000,SPY
359,2024-12-20,581.770020,595.750000,580.909973,591.150024,125716700,1.966,SPY
